#### ***NECESSARY INPUTS FOR THE PROCESS***

In [8]:
import cv2
import numpy as np

#### ***INITIALIZING THE VIDEO***

In [9]:
video_raw:cv2.VideoCapture = cv2.VideoCapture("raw.mp4")
if not video_raw.isOpened():
	raise RuntimeError("Cannot open video")

#### ***INITIALIZING THE PEDESTRIAN DETECTOR***
HOG(Histogram Oriented Descriptor)

In [10]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor.getDefaultPeopleDetector())

In [11]:
ret, frame = video_raw.read()
frame_count = int(video_raw.get(cv2.CAP_PROP_FRAME_COUNT))

frames_per_second = frame_count/120

print(f"{frame_count = }")
print(f"{frames_per_second = }")
print(np.shape(frame))
print(type(frame))

frame_count = 3626
frames_per_second = 30.216666666666665
(1920, 1080, 3)
<class 'numpy.ndarray'>


#### ***DETECTING***

In [12]:
i:int = 0
sensitivity_seconds:int = 12
detected_frames:list = []	# holds the number of detected people per frame
sensitivity:int = int(sensitivity_seconds*frames_per_second)
total_detections:int = 0

while True:
	# ====================
	# reads the next frame of the video
	ret, video_frame = video_raw.read()
	if not ret:
		break  # end of video
	
	# ====================
	# prepares the video for the model
	video_frame_minimized:np.ndarray = cv2.resize(
		video_frame, 
		None, 
		fx=0.125, fy=0.125, 
		interpolation=cv2.INTER_AREA)
	video_frame_gray:np.ndarray = cv2.cvtColor(
		video_frame_minimized, 
		cv2.COLOR_BGR2GRAY)
	
	# model detection varaibles
	(people_detected, _) = hog.detectMultiScale(
		video_frame_gray,
		winStride=(4, 4),
		padding=(4, 4),
		scale=1.2)
	# draw rectangles around people detected
	# for visual debugging
	for (x, y, w, h) in people_detected:
		cv2.rectangle(
			img=video_frame_minimized, 
			pt1=(x, y), 
			pt2=(x + w, y + h), 
			color=(255, 0, 0), 
			thickness=5)
		
	# final detection
	detected_frames.append(people_detected.__len__())
	if len(detected_frames) >= sensitivity:
		print(max(detected_frames))
		total_detections += max(detected_frames)
		detected_frames = []

	# ====================
	cv2.imshow("video", video_frame_minimized)
	if cv2.waitKey(delay = 1) & 0xFF == ord("q"):	# q for early escape 
		break

video_raw.release()
cv2.destroyAllWindows()

1
0
1
1
1
1
0
0
0
1


#### ***RESULTS***

In [ ]:
print(f"{total_detections = }")

total_detections =6
